In [1]:
'''
Iniciamos importando todas las librerias que serán utilies para el desarrollo
del analisis solicitado
**NOTA: Se instala directamente en el notebook la libreria de pandas profiling,
lo recomendado es instalarlo utilizando el gestor de paquetes pip ejecutando
el siguiente comando:
pip install -U pandas-profiling[notebook]
'''
import requests
import json
import urllib
import pandas as pd
!pip install https://github.com/ydataai/pandas-profiling/archive/master.zip
from pandas_profiling import ProfileReport
import sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 21.9 MB 140 kB/s
     |████████████████████████████████| 102 kB 6.0 MB/s 
     |████████████████████████████████| 690 kB 22.0 MB/s 
     |████████████████████████████████| 4.7 MB 8.1 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 812 kB 13.9 MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.2.0-py2.py3-none-any.whl size=261257 sha256=1daadd21305681818f018d1c6cabe58f61791de5d256a65787e94d48f84468ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-tvvi5x98/wheels/c3/aa/10/94ee985ca5a122f047c430cf991df62fe210c4789d3b5762a5
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=3d4d3a70093689f458259ef02754feef7cfd85a27a41b8bd8bb5c3d6710b08ef
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash:

In [2]:
d = {}
def ObtenerApi(anio,mes,dia):
  for x in (n+1 for n in range(dia)):
    url ='http://api.tvmaze.com/schedule/web'
    params= {"date":str(anio)+"-"+str(mes)+"-"+ "{:02d}".format(x)}
    response = requests.get(url, params)

    if response.status_code == 200:
      d[x] = json.loads(response.text)
      d[x] = pd.DataFrame.from_dict(d[x])
      d[x].to_json('File'+str(x)+'.json')
      #print("Export Succesfully")
    else:
      print(f"Error:{response.status_code}")

In [3]:
'''
De acuerdo con la petición se traen todas las
series que se emitieron en diciembre del 2020.
'''
ObtenerApi(2020,12,31)

In [4]:
'''
Normalize data included in the columns
Se normalizan las columnas de los json que vienen como
diccionarios en las columnas del DataFrame
'''
df3 = pd.concat(d.values(), ignore_index = True).reset_index(drop=True)#Une todos los dataframes extraidos en uno solo
df3['rating'] = pd.json_normalize(df3['rating']) #Extrae y reemplaza los valores del diccionario contenido en la columna rating
df4 = pd.json_normalize(df3['image']) #Extrae los valores del diccionario contenido en la columna image y los guarda en un nuevo dataframe
df3['image_medium'] = df4['medium'] # Agrega la columna extraida de image al Dataframe Consolidado
df3['image_original'] = df4['original'] # Agrega la columna extraida de image al Dataframe Consolidado
df3['Extract_links'] = pd.json_normalize(df3['_links']) #Extrae y crea una columna con los valores del diccionario contenido en la columna _links
df5 = pd.json_normalize(df3['_embedded']) #Extrae los valores del diccionario contenido en la columna _embedded y los guarda en un nuevo dataframe
df3 = df3.merge(df5,left_index=True,right_index=True) #Se une el dataframe Consolidado con los valores extraidos de la columna _embedded
df6 = df3.drop(['image','_links','_embedded'], axis=1) # Se eliminan las columnas que contenian diccionarios

In [5]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3061 entries, 0 to 3060
Data columns (total 61 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3061 non-null   int64  
 1   url                               3061 non-null   object 
 2   name                              3061 non-null   object 
 3   season                            3061 non-null   int64  
 4   number                            2980 non-null   float64
 5   type                              3061 non-null   object 
 6   airdate                           3061 non-null   object 
 7   airtime                           3061 non-null   object 
 8   airstamp                          3061 non-null   object 
 9   runtime                           2842 non-null   float64
 10  rating                            302 non-null    float64
 11  summary                           892 non-null    object 
 12  image_

In [6]:
df6.describe()

,id,season,number,runtime,rating,show.id,show.runtime,show.averageRuntime,show.rating.average,show.weight,show.network,show.webChannel.id,show.dvdCountry,show.externals.tvrage,show.externals.thetvdb,show.updated,show.network.id,show.webChannel,show.image,show.webChannel.country
count,3.061000e+03,3061.000000,2980.000000,2842.000000,302.000000,3061.000000,2054.000000,2901.000000,427.000000,3061.000000,0.0,2980.000000,0.0,57.000000,2138.000000,3.061000e+03,213.000000,0.0,0.0,0.0
mean,2.032698e+06,147.568442,23.364765,37.661506,7.949007,46836.475662,38.633885,36.913478,6.855269,35.942829,NaN,145.863758,NaN,23860.263158,359124.135641,1.639108e+09,481.450704,NaN,NaN,NaN
std,9.652817e+04,521.158593,51.625776,30.063129,1.027228,12423.862293,30.780295,28.128825,0.940352,25.996726,NaN,140.224948,NaN,11571.501092,56073.365185,1.859511e+07,527.862523,NaN,NaN,NaN
min,1.732625e+06,1.000000,1.000000,1.000000,3.700000,802.000000,1.000000,1.000000,3.600000,0.000000,NaN,1.000000,NaN,5152.000000,73246.000000,1.602172e+09,8.000000,NaN,NaN,NaN
25%,1.978261e+06,1.000000,4.000000,19.000000,7.300000,44110.000000,20.000000,18.000000,6.600000,16.000000,NaN,21.000000,NaN,19056.000000,345280.000000,1.619513e+09,112.000000,NaN,NaN,NaN
50%,1.990187e+06,1.000000,8.000000,31.500000,7.950000,51928.000000,35.000000,30.000000,7.000000,29.000000,NaN,104.000000,NaN,19056.000000,382296.000000,1.645719e+09,308.000000,NaN,NaN,NaN
75%,2.039619e+06,3.000000,20.000000,45.000000,8.775000,52799.000000,45.000000,45.000000,7.500000,48.000000,NaN,238.000000,NaN,34149.000000,392410.000000,1.654593e+09,551.000000,NaN,NaN,NaN
max,2.380808e+06,2020.000000,358.000000,335.000000,10.000000,63761.000000,300.000000,300.000000,8.800000,100.000000,NaN,533.000000,NaN,47170.000000,419045.000000,1.661808e+09,1862.000000,NaN,NaN,NaN


In [7]:
'''
De acuerdo con la visualización de los datos se eliminan las columnas que no traen
directamente datos o la mayoria de sus datos son nulos dado que estas no tendrán
significancia en el analisis de los datos
'''
df_final = df6.drop(['show.dvdCountry','show.image','show.network','show.webChannel','show.webChannel.country','show.network.officialSite','show.dvdCountry.code','show.dvdCountry.name','show.dvdCountry.timezone','show.externals.tvrage','show._links.nextepisode.href','show.network.country.code','show.network.country.name','show.network.country.timezone','show.network.id','show.network.name'], axis=1)

In [8]:
'''
Coverts string datetime into python date time object
Convertimos todos las fechas que se encuentran como objeto de texto en un formato
de fecha para usar estas en el analisis de datos
'''
df_final['airdate'] = pd.to_datetime(df_final['airdate'], infer_datetime_format=True)  
df_final['show.premiered'] = pd.to_datetime(df_final['show.premiered'], infer_datetime_format=True)
df_final['airstamp'] = pd.to_datetime(df_final['airstamp'], infer_datetime_format=True)
df_final['show.ended'] = pd.to_datetime(df_final['show.ended'], infer_datetime_format=True) 
df_final['Year_show.premiered'] = df_final['show.premiered'].dt.year #Column consists of years when the show was released
#df_final['Year_show.ended'] = df_final['show.ended'].dt.year
#df_final = df_final.reindex(df3.index)

In [9]:
profile = ProfileReport(df_final, minimal=True,title="Profiling Report")#
profile.to_file(output_file="profiling1.html")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3061 entries, 0 to 3060
Data columns (total 46 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   id                                3061 non-null   int64              
 1   url                               3061 non-null   object             
 2   name                              3061 non-null   object             
 3   season                            3061 non-null   int64              
 4   number                            2980 non-null   float64            
 5   type                              3061 non-null   object             
 6   airdate                           3061 non-null   datetime64[ns]     
 7   airtime                           3061 non-null   object             
 8   airstamp                          3061 non-null   datetime64[ns, UTC]
 9   runtime                           2842 non-null   float64      

In [11]:
'''
Despues de analizar los datos se ve que la mayoria posee campos vacios dentro de
la tabla y se procede a rellenar la información 
'''
df_final[['number','airtime','runtime','rating','show.id','show.premiered','show.ended','show.schedule.time','show.rating.average','show.externals.thetvdb','show.externals.imdb','show.updated','Year_show.premiered']] = df_final[['number','airtime','runtime','rating','show.id','show.premiered','show.ended','show.schedule.time','show.rating.average','show.externals.thetvdb','show.externals.imdb','show.updated','Year_show.premiered']].fillna(0)
df_final[['summary','image_medium','image_original','Extract_links','show.url','show.name','show.officialSite','show.webChannel.officialSite','show.image.medium','show.image.original','show.summary','show._links.self.href','show._links.previousepisode.href']] = df_final[['summary','image_medium','image_original','Extract_links','show.url','show.name','show.officialSite','show.webChannel.officialSite','show.image.medium','show.image.original','show.summary','show._links.self.href','show._links.previousepisode.href']].fillna('NI')
df_final[['show.type','show.language','show.status','show.webChannel.name','show.webChannel.country.name','show.webChannel.country.code','show.webChannel.country.timezone']] = df_final[['show.type','show.language','show.status','show.webChannel.name','show.webChannel.country.name','show.webChannel.country.code','show.webChannel.country.timezone']].fillna('OTHER')
df_final[['show.genres','show.schedule.days']] = df_final[['show.genres','show.schedule.days']].fillna('[]')
df_final['show.weight'] = df_final['show.weight'].fillna(method='ffill', inplace=True)
#df_final = df_final[['show.runtime']].fillna(df_final['show.averageRuntime'])
#Season 2020 no exists
# No se puede rellenar un Id ya que este alteraria la referencialidad de las tablas 'show.webChannel.id'

In [12]:
series = df_final[['id','url','name','season','number','type','airdate','airtime','airstamp','runtime','rating','summary','image_medium','image_original','Extract_links']]
series = series.rename(columns = {'number':'series_number'}) 
show = df_final[['id','show.id','show.url','show.name','show.type','show.language','show.genres','show.status','show.runtime','show.averageRuntime','show.premiered','show.ended','show.officialSite','show.schedule.time','show.schedule.days','show.rating.average','show.weight','show.webChannel.id','show.webChannel.name','show.webChannel.country.name','show.webChannel.country.code','show.webChannel.country.timezone','show.webChannel.officialSite','show.externals.thetvdb','show.externals.imdb','show.image.medium','show.image.original','show.summary','show.updated','show._links.self.href','show._links.previousepisode.href','Year_show.premiered']]
show = show.rename(columns = {'id':'id_series'})

In [13]:
%load_ext sql
%sql sqlite:///database.db

database = "/content/database.db"
conn = sqlite3.connect(database)
c = conn.cursor()

In [14]:
c.execute('CREATE TABLE IF NOT EXISTS series (id INT PRIMARY KEY ,url TEXT,name TEXT,season NUMERIC,series_number NUMERIC,type TEXT ,airdate NUMERIC,airtime NUMERIC,airstamp NUMERIC,runtime NUMERIC ,rating NUMERIC,summary TEXT,image_medium TEXT,image_original TEXT,Extract_links TEXT)')
conn.commit()

In [15]:
#c.execute('CREATE TABLE IF NOT EXISTS show (id_series NUMERIC, show.id NUMERIC,show.url TEXT,show.name TEXT,show.type TEXT,show.language TEXT,show.genres TEXT,show.status TEXT,show.runtime INT,show.averageRuntime INT ,show.premiered NUMERIC,show.ended NUMERIC,show.officialSite TEXT,show.schedule.time NUMERIC,show.schedule.days TEXT,show.rating.average NUMERIC,show.weight INT,show.webChannel.id INT,show.webChannel.name TEXT,show.webChannel.country.name TEXT,show.webChannel.country.code TEXT,show.webChannel.country.timezone TEXT,show.webChannel.officialSite TEXT,show.externals.thetvdb INT,show.externals.imdb TEXT,show.image.medium TEXT,show.image.original TEXT,show.summary TEXT,show.updated INT,show._links.self.href TEXT,show._links.previousepisode.href TEXT,Year_show.premiered NUMERIC)')
c.execute('CREATE TABLE IF NOT EXISTS show (id_series NUMERIC, show_id NUMERIC,show_url TEXT,show_name TEXT,show_type TEXT,show_language TEXT,show_genres TEXT,show_status TEXT,show_runtime NUMERIC,show_averageRuntime NUMERIC ,show_premiered NUMERIC,show_ended NUMERIC,show_officialSite TEXT,show_schedule_time NUMERIC,show_schedule_days TEXT,show_rating_average NUMERIC,show_weight NUMERIC,show_webChannel_id NUMERIC,show_webChannel_name TEXT,show_webChannel_country_name TEXT,show_webChannel_country_code TEXT,show_webChannel_country_timezone TEXT,show_webChannel_officialSite TEXT,show_externals_thetvdb NUMERIC,show_externals_imdb TEXT,show_image_medium TEXT,show_image_original TEXT,show_summary TEXT,show_updated NUMERIC,show__links_self_href TEXT,show__links_previousepisode_href TEXT,Year_show_premiered NUMERIC)')
conn.commit()
#FOREIGN KEY(id_series) REFERENCES series(id)

In [16]:
series.to_sql('series', conn, if_exists='replace', index = False)

In [ ]:
show.to_sql('show', conn, if_exists='replace', index = False)

In [ ]:
c.execute('''  
SELECT * FROM series
          ''')

%%sql
select * from series limit 10